In [1]:
from google.colab import drive
drive.mount("drive")

Mounted at drive


In [25]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [26]:
def read_csv(file_path):
  df = pd.read_csv(file_path)
  X = np.array(df["sentence"])
  Y = np.array(df["label"], dtype = int)
  return X, Y

In [27]:
X_train, Y_train = read_csv("/content/drive/MyDrive/Emoji_Text_Classification/train.csv")
X_test, Y_test = read_csv("/content/drive/MyDrive/Emoji_Text_Classification/test.csv")

In [28]:
index = 19
X_train[index], Y_train[index]

('I will celebrate soon', 2)

In [29]:
def label_to_emoji(label):
  emojies = ["❤️","⚽️","😄","😔","🍽"]
  return emojies[label]

In [30]:
index = 19
X_train[index], label_to_emoji(Y_train[index])

('I will celebrate soon', '😄')

In [31]:
unique, counts = np.unique(Y_train, return_counts = True)
print(unique)
print(counts)

[0 1 2 3 4]
[22 19 38 36 17]


In [32]:
max_len = len(max(X_train, key = len).split(" "))
max_len

10

In [33]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip -d glov.6B

In [35]:
f = open("/content/glov.6B/glove.6B.50d.txt", encoding = "utf-8")

word_vectors = {}

for line in f:
  line = line.strip().split()
  word = line[0]
  vector = np.array(line[1:], dtype = np.float64)
  word_vectors[word] = vector

In [ ]:
word_vectors["mindset"]

In [37]:
def sentence_to_avg(sentence):
  try:
    sentence = sentence.lower()
    words = sentence.strip().split(" ")
    sum_vectors = np.zeros((50, ))
    for word in words:
      sum_vectors += word_vectors[word]

    avg_vector = sum_vectors / len(words)

    return avg_vector

  except:
    print(sentence)


In [62]:
mytest = sentence_to_avg("She is ambitious")
mytest.shape

(50,)

In [63]:
X_train_avg = []

for x_train in X_train:
  X_train_avg.append(sentence_to_avg(x_train))

X_train_avg = np.array(X_train_avg)

X_train_avg.shape


(132, 50)

In [ ]:
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes = 5)
Y_train_one_hot

In [65]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(5, input_shape = (50,), activation = "softmax")
])

In [66]:
model.compile (
   tf.keras.optimizers.Adam(),
   loss = "categorical_crossentropy",
   metrics = ["accuracy"]
)


In [67]:
model.fit(X_train_avg, Y_train_one_hot, epochs = 250)

Epoch 1/250
5/5 [==============================] - 0s 4ms/step - loss: 1.6394 - accuracy: 0.2727
Epoch 2/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6198 - accuracy: 0.2803
Epoch 3/250
5/5 [==============================] - 0s 3ms/step - loss: 1.6053 - accuracy: 0.2727
Epoch 4/250
5/5 [==============================] - 0s 4ms/step - loss: 1.5914 - accuracy: 0.2727
Epoch 5/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5785 - accuracy: 0.2955
Epoch 6/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5655 - accuracy: 0.3030
Epoch 7/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5546 - accuracy: 0.2955
Epoch 8/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5441 - accuracy: 0.2879
Epoch 9/250
5/5 [==============================] - 0s 4ms/step - loss: 1.5331 - accuracy: 0.2803
Epoch 10/250
5/5 [==============================] - 0s 3ms/step - loss: 1.5237 - accuracy: 0.3030
Epoch 11/250
5/5 [===========

In [68]:
my_test_avg = np.array([mytest])
result = model.predict(my_test_avg)
y_pred = np.argmax(result)
y_pred
label_to_emoji(y_pred)

1/1 [==============================] - 0s 34ms/step


'😄'

In [73]:
test = sentence_to_avg("She is eating pizza")
test_avg = np.array([test])
result = model.predict(test_avg)
y_pred = np.argmax(result)
y_pred
label_to_emoji(y_pred)

1/1 [==============================] - 0s 17ms/step


'🍽'